<a href="https://colab.research.google.com/github/GEORMC/Nnumerical_Methods_Course/blob/main/The%20coupled%20equation%20in%20the%20time%20domain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

# Define parameters
theta = 0.5  # Time-stepping parameter
delta_t = 0.01  # Time step size
num_steps = 100  # Number of time steps

# Example matrices (replace with your specific problem matrices)
n_dof = 3  # Degrees of freedom (example size)
K_e = np.array([[10, -2, 0], [-2, 10, -2], [0, -2, 10]])  # Example stiffness matrix
Q = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])  # Coupling matrix
S = np.eye(n_dof)  # Storage matrix
H = 0.1 * np.eye(n_dof)  # Hydraulic conductivity matrix

# Initial conditions
u_n = np.zeros(n_dof)  # Initial displacement
p_n = np.zeros(n_dof)  # Initial pore pressure
f_u = np.zeros(n_dof)  # External force
f_p = np.zeros(n_dof)  # Fluid source

# Time-stepping loop
u_history = [u_n]  # Displacement history
p_history = [p_n]  # Pressure history

# Assemble constant matrices
LHS = np.block([
    [theta * K_e, -theta * Q],
    [Q.T, S + delta_t * theta * H]
])

for step in range(num_steps):
    # Calculate RHS
    RHS_u = (1 - theta) * K_e @ u_n + f_u
    RHS_p = S @ p_n - (1 - theta) * delta_t * H @ p_n + delta_t * f_p
    RHS = np.hstack([RHS_u, RHS_p])

    # Solve for next step
    solution = np.linalg.solve(LHS, RHS)
    u_n1, p_n1 = solution[:n_dof], solution[n_dof:]

    # Update history
    u_history.append(u_n1)
    p_history.append(p_n1)

    # Update for next iteration
    u_n, p_n = u_n1, p_n1

# Post-process results
u_history = np.array(u_history)
p_history = np.array(p_history)

# Example: plot results
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(u_history, label="Displacement (u)")
plt.plot(p_history, label="Pore Pressure (p)", linestyle="--")
plt.xlabel("Time Step")
plt.ylabel("Value")
plt.legend()
plt.title("Coupled System Solution")
plt.show()
